# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [25]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
link_system_prompt += "Another example would be the following:"
link_system_prompt += """
{
    "links": [
        {"type": "chat page", "url": "https://siteurl/chat"}
    ]
}
"""

In [26]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
Another example would be the following:
{
    "links": [
        {"type": "chat page", "url": "https://siteurl/chat"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [27]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [28]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nanonets/Nanonets-OCR-s',
 '/google/magenta-realtime',
 '/MiniMaxAI/MiniMax-M1-80k',
 '/mistralai/Mistral-Small-3.2-24B-Instruct-2506',
 '/Menlo/Jan-nano',
 '/models',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/enzostvs/deepsite',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces/OmniGen2/OmniGen2',
 '/spaces/multimodalart/self-forcing',
 '/spaces',
 '/datasets/EssentialAI/essential-web-v1.0',
 '/datasets/institutional/institutional-books-1.0',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/AceReason-1.1-SFT',
 '/datasets/nvidia/OpenScience',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers'

In [29]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [33]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    # print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
nanonets/Nanonets-OCR-s
Updated
5 days ago
•
177k
•
1.14k
google/magenta-realtime
Updated
2 days ago
•
290
MiniMaxAI/MiniMax-M1-80k
Updated
1 day ago
•
10.4k
•
566
mist

In [44]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Give a clever title to the \
brochure at the top of the response. At the end, crack a funny joke and show the contact details."

In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nnanonets/Nanonets-OCR-s\nUpdated\n5 days ago\n•\n177k\n•\n1.14k\ngoogle/magenta-realtime\nUpdated\n2 days ago\n•\n290\nMiniMaxAI/MiniMax-M1-80k\nUpdated\n1 day ago\n•\n10.4k\n•\n566\nmistralai/Mistral-Small-3.2-24B-Instruct-2506\nUpdated\n3 days ago\n•\n5.37k\n•\n228\nMenlo/Jan-nano\nUpdated\n8 days ago\n•\n29.1k\n•\n399\nBrowse 1M+ models\nSpaces\nRunning\n796\n796\nSparc3D\n🏃\nNext-Gen H

In [32]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [34]:
create_brochure("HuggingFace", "https://huggingface.co")

# Hugging Face: Embrace the Future of AI

## About Us
Welcome to Hugging Face, the AI community dedicated to building the future of machine learning through collaboration and innovation. With over **1 million models** and **250,000 datasets**, our platform offers an extensive collection of resources designed for everyone from hobbyists to enterprise-level organizations. Join us in transforming how AI is developed and implemented!

## Our Mission
At Hugging Face, we are on a mission to democratize machine learning—one commit at a time. We believe in the power of open-source solutions that empower the community, enabling everyone to create, discover, and collaborate more effectively.

## Why Choose Us?
- **Community-Driven:** We are an enthusiastic community that fosters collaboration among developers, researchers, and organizations. More than **50,000 organizations** – including tech giants like Google, Microsoft, and Amazon – trust us to host their models.
- **Diverse Offerings:** From state-of-the-art ML models using **Transformers** to **Diffusers** for advanced graphics generation, our resources cover all aspects of machine learning, including text, image, video, audio, and 3D.
- **Enterprise Solutions:** Take advantage of our premium solutions featuring enterprise-grade security, dedicated support, and the flexibility of a **pay-as-you-go** pricing model.

## Company Culture
At Hugging Face, we prioritize an inclusive and innovative work environment. Our team consists of over **213 individuals** who collaborate across various projects and embrace diversity in perspectives and skills. We are passionate about making AI accessible and ethical for all.

## Explore Career Opportunities
Are you ready to join a dynamic team pushing the boundaries of AI? We are always looking for talent in different areas. Whether you are an engineer, researcher, or someone with a passion for AI, come and build the future with us!

### Open Positions
Check our **[Careers page](https://huggingface.co/jobs)** for current opportunities and see how you can contribute to our exciting mission.

## Connect with Us
Interested in learning more about Hugging Face or have a question? Reach out today:

- **Website:** [huggingface.co](https://huggingface.co)
- **Email:** info@huggingface.co
- **Social Media:** Follow us on GitHub, Twitter, LinkedIn, and Discord.

## A Little Humor
Why did the computer keep freezing?  
Because it left its Windows open! 😄

Ready to take the leap into the world of machine learning with hugging Face? Let's get started!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure 

## Welcome to Hugging Face
### The AI Community Building the Future

At Hugging Face, we are dedicated to creating an open and collaborative platform for the machine learning community. Our mission is to provide tools and resources that facilitate shared knowledge, accelerate innovation, and empower individuals and organizations to explore the vast opportunities in artificial intelligence.

### What We Offer
- **Models & Datasets:** We host over **1 million models** and **250,000 datasets** that users can browse, discover, and contribute to. This extensive library includes popular models like `nanonets/Nanonets-OCR-s` and `google/magenta-realtime`, allowing seamless access to cutting-edge machine learning technologies.
  
- **Spaces:** A dynamic environment where users can run applications like high-resolution 3D model generation and applications that generate code from text prompts. Our hosted **Spaces** ensure you can easily build and showcase your AI applications.

- **Enterprise Solutions:** We provide specialized services tailored for enterprises, offering enhanced security, priority support, and dedicated infrastructures starting from $20/user/month.

### Our Community
With more than **50,000 organizations** utilizing our resources, including giants like **Meta, Amazon, Google, and Microsoft**, Hugging Face serves a diverse clientele. We believe that through collaboration and community engagement, we can push the boundaries of what is possible in AI. 

### Company Culture
At Hugging Face, we champion an inclusive and open culture that encourages collaboration, creativity, and continuous learning. We pride ourselves on our community-driven approach where everyone's contributions are valued. Our workplace aligns with modern ideals, emphasizing remote engagement and flexibility to ensure we attract and retain top talent.

### Join Us
- **Careers:** Embrace an exciting career at Hugging Face! We are always looking for passionate individuals in machine learning, engineering, data science, and business development who want to make a difference. Here, you'll have the chance to work on state-of-the-art technologies and be part of a community that is shaping the future of AI.
- **Contributing to Open Source:** Engage with our open-source projects. We provide numerous opportunities for developers to collaborate, innovate, and share their findings with the global ML community.

### Get Involved
Ready to explore the power of AI with us? Sign up today to start your journey, collaborate with others, and contribute to the AI landscape transforming the future.

[Explore Models & Datasets](https://huggingface.co/models) | [Join the Community](https://huggingface.co/join) | [Learn More About Careers](https://huggingface.co/jobs)

---

**Hugging Face — Together, Building the Future of AI!**

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Welcome to Hugging Face! 🤗

## The AI Playground Where Models Make Friends

---
### Who Are We?

At Hugging Face, we’re like that charismatic friend who introduces you to all the cool folks at a party—except our party is all about machine learning! We are the **AI community building the future**, tackling the tough questions like, "How many models can fit on a server?" and "Is there anything cuter than a puppy model?" 

Wait, don’t answer that last one! 🐶

---

### What Do We Do?

**Models**, **Datasets**, and **Spaces**—oh my!  

- **1 Million+ Models:** That's right! It's like the model casting call for the next big hit, except it's all AI! 🎭
- **250k+ Datasets:** We’ve got a dataset for every occasion: weddings, birthdays, even that time you tried to make a cake but ended up with a pancake. 🎂
- **Spaces:** A place where creativity thrives! Think of it as the AI startup incubator, minus the fancy coffee. ☕

### Trending Models This Week:
1. **Nanonets OCRs** - They’ll read the fine print while you sip on your espresso. ☕
2. **Google’s Magenta** - The musical genius your last band was missing. 🎤
3. **Mistral Small** - A compact powerhouse. Definitely not a superhero sidekick! 🦸

---

### Join the Community! 

Why join Hugging Face? Because we’re all about **collaboration**! Whether you're a **data scientist**, **AI wizard**, or just someone who likes to play with models, there’s a place for you here.   
- 🤝 Collaborate with **50,000+ organizations** including Google, Microsoft, and the friendly neighborhood Amazon.   
- 📈 Share your work and pass around high-fives like they’re going out of style!  

---

### Company's Culture: Less Stress, More "Yes!"

At Hugging Face, our motto is “Democratize machine learning, one awkward high-five at a time!” High-fives are optional but highly encouraged. Here, we believe in:
- **Open Source Freedom:** Share your ideas without having to pay a toll!
- **Flexibility:** Work remotely in your pajamas or be that overachiever who shows up in business casual. Your choice! 👖
- **Supportive Environment:** Always ready to jump in and help—unless it’s a spider in the office, we draw the line there! 🕷️

---

### Careers: Join Us! 🧑‍💻

Are you a champion of authenticity, an enthusiast of *all things AI*, and looking for a place where your genius can flourish? Well, grab your cape (or laptop) and check out our open **job positions**! 
- **Perks include:** Unlimited snacks, countless models, and expert-level machine learning puns that will make you sigh and chuckle at the same time! 

### Final Note

So, whether you’re a fellow AI enthusiast, a potential investor looking for the next big thing, or a job seeker ready to change the world (and enjoy a few laughs), remember that at Hugging Face—we're not just building models, we're building a community! 

**Ready to warm up and join the fun?**   
Let’s start hugging the future, one algorithm at a time! 🤗💻

---

> Visit us at [HuggingFace.co](https://huggingface.co) and let’s get this conversational AI party started!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

## Spanish Challenge

In [35]:
def return_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [36]:
print(return_brochure("HuggingFace", "https://huggingface.co"))

# HuggingFace: Embracing the Future of AI

## About Us
At HuggingFace, we're not just a company; we're a thriving community dedicated to shaping the future of artificial intelligence. As the central hub for every aspect of machine learning—ranging from models to datasets to applications—we empower individuals and organizations to collaborate, innovate, and progress together.

## Our Offerings
- **Models**: Browse over **1 million** models, including the latest in AI development.
- **Datasets**: Access a comprehensive library of **250,000+ datasets** tailored for every ML task imaginable.
- **Spaces**: Create and share **400,000+ applications** encompassing all modalities, whether text, image, video, or 3D.
- **Enterprise Solutions**: Our robust infrastructure supports over **50,000 organizations**, from start-ups to industry leaders such as Amazon, Google, and Microsoft, providing advanced tools with enterprise-grade security and dedicated support.

## Company Culture
At HuggingFace, i

In [38]:
spanish_system_prompt = "You are an assistant that translates the contents of a brochure from english to spanish. \
Make sure the brochure is responded in markdown and do your best to translate."

In [45]:
spanish_user_prompt = "You are given the following brochure to translate into Spanish from English:\n"
spanish_user_prompt += return_brochure("HuggingFace", "https://huggingface.co")

In [46]:
spanish_user_prompt

"You are given the following brochure to translate into Spanish from English:\n# Embrace the Future with Hugging Face! \n\n## Welcome to Hugging Face\nAt Hugging Face, we're more than just a company; we're a vibrant community dedicated to building the future of artificial intelligence together. Our platform fosters collaboration among machine learning enthusiasts, researchers, and enterprises alike, empowering everyone from hobbyists to industry leaders.\n\n## Our Offerings\n- **Models**: Browse over 1 million cutting-edge machine learning models and harness the power of AI for your projects.\n- **Datasets**: Access a vast library of 250,000+ datasets tailored for every machine learning task.\n- **Spaces**: Create and share AI applications effortlessly using our intuitive platform.\n\n## A Thriving Community\nJoin our growing community of over 50,000 organizations, including industry giants like Google, Amazon, Microsoft, and Grammarly. Our collective goal is to democratize machine lea

In [47]:
response = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": spanish_system_prompt},
        {"role": "user", "content": spanish_user_prompt}
      ],
)
result = response.choices[0].message.content

In [51]:
display(Markdown(result))

```markdown
# ¡Abraza el Futuro con Hugging Face!

## Bienvenido a Hugging Face
En Hugging Face, somos más que una empresa; somos una comunidad vibrante dedicada a construir el futuro de la inteligencia artificial juntos. Nuestra plataforma fomenta la colaboración entre entusiastas del aprendizaje automático, investigadores y empresas por igual, empoderando a todos, desde aficionados hasta líderes de la industria.

## Nuestras Ofertas
- **Modelos**: Explora más de 1 millón de modelos de aprendizaje automático de vanguardia y aprovecha el poder de la IA para tus proyectos.
- **Conjuntos de Datos**: Accede a una vasta biblioteca de más de 250,000 conjuntos de datos adaptados para cada tarea de aprendizaje automático.
- **Espacios**: Crea y comparte aplicaciones de IA sin esfuerzo utilizando nuestra plataforma intuitiva.

## Una Comunidad en Crecimiento
Únete a nuestra creciente comunidad de más de 50,000 organizaciones, incluidas gigantes de la industria como Google, Amazon, Microsoft y Grammarly. Nuestro objetivo colectivo es democratizar el aprendizaje automático a través de la colaboración e innovación de código abierto.

## Cultura Empresarial
En Hugging Face, creemos que la colaboración y la creatividad son la clave del éxito. Celebramos una cultura donde cada voz importa y cada contribución cuenta hacia nuestra misión. Nuestro entorno amigable para el trabajo remoto promueve la flexibilidad, la inclusión y el compromiso, asegurando que todos puedan prosperar.

### Carreras en Hugging Face
¿Estás listo para llevar tu carrera al siguiente nivel con un equipo pionero que está redefiniendo la inteligencia artificial? Siempre estamos en busca de individuos apasionados para unirse a nosotros en diversas funciones. Ya sea que te interese la ingeniería de software, la investigación o el marketing, hay un lugar para ti en nuestra misión de democratizar el aprendizaje automático.

Visita nuestra [Página de Carreras](https://huggingface.co/jobs) para explorar las oportunidades laborales actuales.

## Alianzas para el Éxito
Experimenta el poder de la adaptabilidad de la IA con nuestras soluciones empresariales, diseñadas para proporcionar plataformas avanzadas equipadas con seguridad de primer nivel y soporte dedicado. ¡Empieza a construir tu IA con nosotros hoy!

## ¡Conectémonos!
Para más información o para convertirse en parte de nuestra comunidad, contáctanos:

- **Sitio Web**: [Hugging Face](https://huggingface.co)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face](https://www.linkedin.com/company/huggingface)

---

**¿Por qué rompió el ordenador con Internet?**  
¡Porque no pudo encontrar una conexión!

¡Esperamos saber de ti pronto!
```